In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Pancreatic_Cancer"
cohort = "GSE157494"

# Input paths
in_trait_dir = "../../input/GEO/Pancreatic_Cancer"
in_cohort_dir = "../../input/GEO/Pancreatic_Cancer/GSE157494"

# Output paths
out_data_file = "../../output/preprocess/Pancreatic_Cancer/GSE157494.csv"
out_gene_data_file = "../../output/preprocess/Pancreatic_Cancer/gene_data/GSE157494.csv"
out_clinical_data_file = "../../output/preprocess/Pancreatic_Cancer/clinical_data/GSE157494.csv"
json_path = "../../output/preprocess/Pancreatic_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Improved patient-derived tumor models in pancreatic ductal adenocarcinoma employing orthotopic implantation"
!Series_summary	"Pancreatic ductal adenocarcinoma has a very poor prognosis, and new therapies and preclinical models are urgently needed. We developed patient-derived xenografts (PDXs), established PDX-derived cell lines (PDCLs), and generated cell line-derived xenografts (CDXs), and integrated these to create 13 matched trios, as systematic models for this cancer. Orthotopic implantation (OI) of PDCLs showed tumorigenesis and metastases to the liver and peritoneum. Morphological comparisons of OI-CDX and OI-PDX with passaged tumors showed that histopathological features of the original tumor were maintained in both models. Molecular alterations in PDX tumors (including those to KRAS, TP53, SMAD4, and CDKN2A) were similar to those in the respective PDCLs and CDX tumors. Comparing gene expression in PDCLs, ectopic tumors, and OI tumors, CXC

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import json
import os
from typing import Optional, Callable, Dict, Any, List, Union

# Checking available data 
is_gene_available = True  # Based on series summary, they used Affymetrix Gene Chip HG-U133Plus2.0

# 2.1 Data Availability
# Sample characteristics dictionary shows sample types but no direct trait, age, or gender info
# Since this is a patient-derived xenograft study, we consider trait as the cancer status
# From context, we know all samples are pancreatic cancer (either xenografts or cell lines)
trait_row = 0  # Using the sample type row to determine if it's a tumor or cell line
age_row = None  # Age information is not available
gender_row = None  # Gender information is not available

# 2.2 Data Type Conversion
def convert_trait(value: str) -> int:
    """Convert sample type to binary trait (1 for tumor, 0 for cell line)"""
    if value is None:
        return None
    
    # Extract the value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Map the sample type to binary values
    if 'xenografted tumor' in value.lower():
        return 1  # Tumor
    elif 'cell line' in value.lower():
        return 0  # Cell line
    else:
        return None  # Unknown

def convert_age(value: str) -> Optional[float]:
    """Convert age to float (not used as age is not available)"""
    return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender to binary (not used as gender is not available)"""
    return None

# 3. Save Metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Only execute if trait_row is not None
if trait_row is not None:
    # We assume clinical_data is already available from previous step
    try:
        # Try to access clinical_data (assuming it was created in previous steps)
        clinical_data
    except NameError:
        # If clinical_data is not defined, create an empty DataFrame with the structure from sample characteristics
        clinical_data = pd.DataFrame({
            0: ['sample type: xenografted tumor', 'sample type: Cell line']
        })
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview_data = preview_df(selected_clinical_df)
    print("Preview of clinical data:")
    print(preview_data)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical data
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of clinical data:
{'GSM4767149': [1.0], 'GSM4767150': [1.0], 'GSM4767151': [1.0], 'GSM4767152': [1.0], 'GSM4767153': [1.0], 'GSM4767154': [1.0], 'GSM4767155': [1.0], 'GSM4767156': [1.0], 'GSM4767157': [1.0], 'GSM4767158': [1.0], 'GSM4767159': [1.0], 'GSM4767160': [1.0], 'GSM4767161': [1.0], 'GSM4767162': [1.0], 'GSM4767163': [0.0], 'GSM4767164': [0.0], 'GSM4767165': [0.0], 'GSM4767166': [0.0], 'GSM4767167': [0.0], 'GSM4767168': [0.0], 'GSM4767169': [0.0], 'GSM4767170': [0.0], 'GSM4767171': [0.0], 'GSM4767172': [0.0], 'GSM4767173': [0.0], 'GSM4767174': [0.0], 'GSM4767175': [0.0], 'GSM4767176': [1.0], 'GSM4767177': [1.0], 'GSM4767178': [1.0], 'GSM4767179': [1.0], 'GSM4767180': [1.0], 'GSM4767181': [1.0], 'GSM4767182': [1.0], 'GSM4767183': [1.0], 'GSM4767184': [1.0], 'GSM4767185': [1.0], 'GSM4767186': [1.0], 'GSM4767187': [1.0], 'GSM4767188': [1.0], 'GSM4767189': [1.0]}
Clinical data saved to ../../output/preprocess/Pancreatic_Cancer/clinical_data/GSE157494.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Analyze the gene identifiers
# The identifiers like '1007_s_at', '1053_at', '117_at', etc. are Affymetrix probe IDs
# from the HG-U133 series microarray platform, not standard human gene symbols
# These need to be mapped to human gene symbols for proper analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Analyze the gene annotation data to identify the columns for probe IDs and gene symbols
# From the preview, we can see that the column 'ID' contains the probe IDs (e.g., '1007_s_at')
# The column 'Gene Symbol' contains the gene symbols (e.g., 'DDR1 /// MIR4640')

# 2. Use the get_gene_mapping function to extract the mapping between probe IDs and gene symbols
gene_mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
# This handles the many-to-many relationships between probes and genes
gene_data = apply_gene_mapping(gene_data, gene_mapping_df)

# Print the shape of the resulting gene expression dataframe and preview the first few gene symbols
print(f"Shape of gene expression data after mapping: {gene_data.shape}")
print("First 10 gene symbols after mapping:")
print(gene_data.index[:10])


Shape of gene expression data after mapping: (21278, 41)
First 10 gene symbols after mapping:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1',
       'A4GALT', 'A4GNT', 'AA06'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Verify we have the clinical_data from earlier steps
try:
    # Load the saved clinical data if it exists
    clinical_df = pd.read_csv(out_clinical_data_file, index_col=0)
    print(f"Loaded clinical data from file with shape: {clinical_df.shape}")
except:
    # Use the clinical data extracted in the previous steps
    print(f"Using clinical data from previous steps with shape: {selected_clinical_df.shape}")
    clinical_df = selected_clinical_df

# 2. Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# 3. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Shape after handling missing values: {linked_data.shape}")

# 4. Determine whether the trait and demographic features are severely biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains pancreatic cancer xenografted tumors and cell lines."
)

# 6. If the linked data is usable, save it as a CSV file
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    # Save the data
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data quality check failed. Linked data not saved.")

Normalized gene data saved to ../../output/preprocess/Pancreatic_Cancer/gene_data/GSE157494.csv
Loaded clinical data from file with shape: (1, 41)
Linked data shape: (41, 19846)


Shape after handling missing values: (41, 19846)
For the feature 'Pancreatic_Cancer', the least common label is '0.0' with 13 occurrences. This represents 31.71% of the dataset.
The distribution of the feature 'Pancreatic_Cancer' in this dataset is fine.



Linked data saved to ../../output/preprocess/Pancreatic_Cancer/GSE157494.csv
